In [148]:
#Importing necessary libraries and frameworks
import tensorflow as tf
import scipy as sc
import numpy as np
import time
import sklearn.preprocessing
from scipy.signal import butter,lfilter


In [149]:
#Defining a butterworth filtter bandpass
def butter_bandpass(lowcut,highcut,fs,order=5):
    #Nyquist rate fs=2*fm
    #fm=0.5*fs
    nyq=0.5*fs
    lowcut=lowcut/nyq
    highcut=highcut/nyq
    #b,a=numerator and denominator values of the IIR Filter
    b,a=butter(order,[lowcut,highcut],btype='band')
    return b,a

In [150]:
#Defining one hot encoding
#This function is used to transfer one column label to one hot label
def one_hot(y_):
    #Function to encode output labels from number indexes
    #e.g [[5],[3],[0]] --> [[0,0,0,0,0,1],[0,0,0,1,0,0],[0,0,0,0,0,0]]
#     y_=y_.reshape((len(y_)))
    y_=np.array(y_)
    y_=y_.reshape(len(y_))
    n_values=np.max(y_)+1
    return np.eye(int(n_values))[np.array(y_,dtype=np.int32)]

In [151]:
#Checking
a1=[[5],[3],[2]]
a2=one_hot(a1)
print(a2)

[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [152]:
def butter_bandpass_filter(data,lowcut,highcut,fs,order=5):
    b,a=butter_bandpass(lowcut,highcut,fs,order=order)
    y=lfilter(b,a,data)
    return y
    

Defining some variables used to load and process the EEG Data
1. **len_sample** = Length of each sample ( in seconds )
2. **full**= total number of samples per subject 
3. **n_fea**=number of features ( EEG Channels )
4. **label0-label7** = Defines the labels for each sample

In [153]:
len_sample=1
full=7000
len_a=int(full/len_sample)
label0=np.zeros(len_a)
label1=np.ones(len_a)
label2=np.ones(len_a)*2
label3=np.ones(len_a)*3
label4=np.ones(len_a)*4
label5=np.ones(len_a)*5
label6=np.ones(len_a)*6
label7=np.ones(len_a)*7
label=np.hstack((label0,label1,label2,label3,label4,label5,label6,label7))
label=label.T
label.shape=(len(label),1)
print(label)

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [154]:
start1=time.process_time()

In [155]:
feature=sc.io.loadmat("EID-S.mat")
all=feature['eeg_close_8sub_1file']

In [156]:
n_fea=14
all=all[0:full*8,0:n_fea]

### Next steps - Performing a filtering operation on a EEG Dataset to extract the delta wave pattern using a Butterworth filter

1. **start2**= measures the starting time using `time.process_time()` function and stores it in a variable
2. Initialize an empty list **data_f1** to store the filtered EEG data
3. Loop over the columns of the `all` array using `range(all.shape[1])`
4. For each column, applies a bandpass filter using the butter_bandpass_filter function with the specified lowcut, highcut and sampling frequency values. The filtered data is then stored in the `data_f1 list`.
5. Convert the `data_f1` list to a numpy array and then transpose it
6. Replace the original `all` array with the filtered data array `data_f1`
7. Measure the ending time using the `time.process_time()` function and store it in a variable `time3`
8. Print the shape of the filtered data array and the elapsed timem for the filtering operation

In [157]:
#Timing the process
start=time.process_time()

#Empty list
data_f1=[]
for i in range(all.shape[1]):
    x=all[:,i]
    fs=128.0
    lowcut=0.5
    highcut=4.0
    y=butter_bandpass_filter(x,lowcut,highcut,fs,order=3)
    data_f1.append(y)

In [158]:
data_f1=np.array(data_f1)
data_f1=data_f1.T
print('data_f1.shape',data_f1.shape)

data_f1.shape (56000, 14)


In [159]:
all=data_f1
end=time.process_time()
print('PD time',start-end)
all=np.hstack((all,label))
print(all.shape)

PD time 0.0
(56000, 15)


In [160]:
#all=combine data
data_size=all.shape[0]

feature_all=all[:,0:n_fea]
print(all[:,n_fea:n_fea+1])

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [161]:
feature_all=feature_all-4200 #minus Direct Current
#z-score scaling
feature_all=sklearn.preprocessing.scale(feature_all)

In [162]:
label_all=all[:,n_fea:n_fea+1]
all=np.hstack((feature_all,label_all))
print(all.shape)

(56000, 15)


In [163]:
#Using the first subject as testing subject
np.random.shuffle(all)

In [164]:
train_data=all[0:int(data_size*0.875)] #Training data is 87.5% of original
test_data=all[int(data_size*0.875):data_size]

no_fea=n_fea
n_steps=len_sample

feature_training=train_data[:,0:no_fea]
feature_training=feature_training.reshape([train_data.shape[0],n_steps,no_fea])

feature_testing=test_data[:,0:no_fea]
feature_training = feature_training.reshape([train_data.shape[0], n_steps, no_fea])


feature_testing = test_data[:,0:no_fea]

feature_testing = feature_testing.reshape([test_data.shape[0], n_steps, no_fea])

In [165]:
label_training=train_data[:,no_fea]
label_training=one_hot(label_training)
label_testing=test_data[:,no_fea]
print(label_testing)

[0. 1. 1. ... 7. 3. 6.]


In [166]:
print(all.shape)

(56000, 15)


### Batch Splitting
**Splitting the input data into batches for training in order to process it more efficiently**

***Steps Involved***
1. It calculates the batch size by multiplying the total data size by 0.125 (12.5%).
2. It creates an empty list called train_fea.
3. It sets the number of groups to 7.
4. It loops through each group and selects the corresponding slice of the feature_training array to add to the train_fea list. Each slice is determined by multiplying the batch size by the group index and adding 0 to the start and end indices for the first group, adding one batch size to both the start and end indices for the second group, and so on. The result is that train_fea is a list of 7 arrays, each with batch_size rows and some number of columns.
5. It prints the shape of the first element of train_fea (which should be (batch_size, num_features)) and the elements of rows 235 and 236 of the first element of train_fea.
6. It creates an empty list called train_label.
7. It loops through each group and selects the corresponding slice of the label_training array to add to the train_label list. The slices are determined in the same way as for train_fea. The result is that train_label is a list of 7 arrays, each with batch_size rows and some number of columns.
8. It prints the shape of the first element of train_label (which should be (batch_size, num_classes)). 


In [167]:
a=feature_training
b=feature_testing
nodes=30
lameda=0.001
lr=0.001

batch_size=int(data_size*0.125)

n_group=7

def range_calculator(listname,listdest):
    for i in range(n_group):
        f=listname[(0+batch_size*i):(batch_size+batch_size*i)]
        listdest.append(f)

train_fea=[]
train_label=[]

range_calculator(a,train_fea)
range_calculator(label_training,train_label)

print(train_fea[0].shape)
print(train_fea[0][235:237])

print(train_label[0].shape)

(7000, 1, 14)
[[[ 0.042783    0.1377656  -0.01978495  0.00890953 -0.0072165
    0.06600291 -0.03770078 -0.01082981 -0.07334202  0.03269157
    0.06984906  0.00142352  0.04783427  0.01854654]]

 [[-0.17298104 -0.07393439 -0.03212914  0.19396801 -0.09174386
    0.16028462  0.0908069  -0.0637945  -0.1560574  -0.14098168
    0.00060067 -0.02152958 -0.13572034 -0.16545925]]]
(7000, 8)


### Defining an RNN Cell 

In [168]:
#hyperparameters
n_inputs=no_fea
n_hidden1_units=nodes
n_hidden2_units=nodes
n_hidden3_units=nodes
n_hidden4_units=nodes
n_classes=8

#tf graph input 
x=tf.keras.layers.Input(shape=(None,n_inputs),name="x",dtype='float32')
y=tf.keras.layers.Input(shape=(n_classes),dtype='float32',name="y")

In [169]:
#Defining weights
weights = {
    'in': tf.Variable(tf.random.normal([n_inputs, n_hidden1_units]), trainable=True),
    'a': tf.Variable(tf.random.normal([n_hidden1_units, n_hidden1_units]), trainable=True),
    'hidd2': tf.Variable(tf.random.normal([n_hidden1_units, n_hidden2_units])),
    'hidd3': tf.Variable(tf.random.normal([n_hidden2_units, n_hidden3_units])),
    'hidd4': tf.Variable(tf.random.normal([n_hidden3_units, n_hidden4_units])),
    'out': tf.Variable(tf.random.normal([n_hidden4_units, n_classes]), trainable=True),
    'att': tf.Variable(tf.random.normal([n_inputs, n_hidden4_units]), trainable=True),
    'att2': tf.Variable(tf.random.normal([1, batch_size]), trainable=True),
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden1_units])),
    'hidd2': tf.Variable(tf.constant(0.1, shape=[n_hidden2_units])),
    'hidd3': tf.Variable(tf.constant(0.1, shape=[n_hidden3_units])),
    'hidd4': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes]), trainable=True),
    'att': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
    'att2': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
}

In [170]:
def RNN(X,weights,biases):
    #hidden layer for input to cell
    ##############################
    
    X=tf.reshape(X,[-1,n_inputs])
    
    #hidden layer
    X_hidd1=tf.sigmoid(tf.matmul(X,weights['in'])+biases['in'])
    X_hidd2=tf.matmul(X_hidd1,weights['hidd2'])+biases['hidd2']
    X_hidd3=tf.matmul(X_hidd2,weights['hidd3'])+biases['hidd3']
    
    X_in=tf.reshape(X_hidd3,[-1,n_steps,n_hidden4_units])
    
    
    #cell
    #####################################
    
    #basic LSTM Cell
    lstm_cell_1=tf.keras.layers.LSTMCell(n_hidden3_units,unit_forget_bias=True)
    init_state=lstm_cell_1.get_initial_state(batch_size=batch_size,dtype=tf.float32)
    
    outputs,final_state=tf.keras.layers.RNN(lstm_cell_1,return_sequences=True,return_state=True)(X_in,initial_state=init_state)
    
    #outputs
    outputs=tf.unstack(tf.transpose(outputs,[1,0,2])) #states is the last output
    
    #attention based model
    X_att2=final_state[0] #weights
    outputs_att=tf.multiply(outputs[-1],X_att2)
    results=tf.matmul(outputs_att,weights['out'])+biases['out']
    
    return results,outputs[-1]


pred,Feature,_=RNN(x,weights,biases)
    

The following Variables were used a Lambda layer's call (tf.linalg.matmul_15), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(14, 30) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.__operators__.add_15), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(30,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.linalg.matmul_16), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(30, 30) dtype=float32>
It is possible that this is intended behavior, but it is more 

ValueError: too many values to unpack (expected 2)

In [ ]:
lamena=lameda
#Computing l2 loss
l2=lamena*sum(tf.nn.l2_loss(tf_var) for tf_var in tf.compat.v1.trainable_variables())
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))+l2